In [1]:
import pandas as pd

from tqdm.notebook import tqdm
df = pd.read_csv('dataset/raw_dataset.csv')
df.sort_values(by=['created_at'], inplace=True)
print(df.head())

tqdm.pandas()

             id                                  change_id  \
735913  1104031  I4efe55a84645a30f66f413cd614e5387582577aa   
735914  1104032  I4efe55a84645a30f66f413cd614e5387582577aa   
293385   661203  I32e56a9b18b5feeb41b583081e95421cf1984522   
293386   661204  I32e56a9b18b5feeb41b583081e95421cf1984522   
293387   661205  I32e56a9b18b5feeb41b583081e95421cf1984522   

                                      message_id  author  real_author  \
735913  fd006a75943b16c9ae1bddc497adb04f5fefd2fa    1782         1782   
735914  2611482eaf227257feec82a61631547293d82aa2    1782         1782   
293385  a283141cc95dfd068c13845eeb579a6b0bc422ec    1176         1176   
293386  050d7f37c0f9f6d87a15e538967729c38a387006    1176         1176   
293387  b475141a088f5d838311aca15e4eb50a50ad1f45    1176         1176   

                           created_at                message  revision_number  \
735913  2012-08-02 19:09:55.000000000  Uploaded patch set 2.                2   
735914  2012-08-02 19:11:1

In [2]:
# table II number of recheck after CI fails

# at the code review level

total_build_pass = 0
total_recheck = 0

for change_id, group in tqdm(df.groupby(['change_id'])):
    build_fail = False
    recheck = False
    build_pass = False
    recheck_after_build_fail = False
    
    for ind, col in group.iterrows():
        
        if 'build failed' in col['message'].lower() and col['is_bot']:
            build_fail = True
        
        if 'build succ' in col['message'].lower() and col['is_bot']:
            build_pass = True
        
        if not build_pass:
            continue
            
        if 'recheck' in col['message'].lower() and not col['is_bot']:
            recheck = True
            
            if build_fail:
                recheck_after_build_fail = True
                
    if recheck_after_build_fail:
        continue
            
    if build_pass:
        total_build_pass += 1
    
    if recheck:
        total_recheck += 1
        
print(f"Code Reviews # With Passing CI jobs: {total_build_pass}, Invoked Rechecks: {total_recheck} ({round(total_recheck * 100 / total_build_pass)})%")
      

  0%|          | 0/66932 [00:00<?, ?it/s]

Code Reviews # With Passing CI jobs: 43243, Invoked Rechecks: 1003 (2)%


In [3]:
# table II number of recheck after CI fails at the patch set level

# Fig 2 Proportion of patch sets and number of invoked rechecks
# count how many recheck after build fail

## calculate number of them that shown gate jobs

total_build_pass = 0
total_recheck = 0
total_gate_job = 0
total_gate_job_after_recheck = 0
recheck_list = []
other_count = 0
gate_job_recheck_list = []
pass_build_recheck_list = []

for change_id, group in tqdm(df.groupby(['change_id','revision_number'])):
    build_fail = False
    recheck_count = 0
    build_pass = False
    gate_job = False

    for ind, col in group.iterrows():
        
        if 'build failed' in col['message'].lower() and col['is_bot']:
            build_fail = True
        
        if 'build succ' in col['message'].lower() and col['is_bot']:
            build_pass = True
        
        if 'gate job' in col['message'].lower() and col['is_bot']:
            gate_job = True
        
        if not build_pass:
            continue
            
        if 'recheck' in col['message'].lower() and not col['is_bot']:
            recheck_count += 1

    if not build_fail and not build_pass:
        other_count += 1
                
    if build_fail:
        continue
                    
    if build_pass:
        total_build_pass += 1

    
    if recheck_count:
        total_recheck += 1
        recheck_list.append(recheck_count)
        pass_build_recheck_list.append(change_id)
        
        if gate_job:
            total_gate_job += 1
            gate_job_recheck_list.append(change_id)
            
print(f"Patch sets # With Failing CI jobs: {total_build_pass}, Invoked Rechecks: {total_recheck} ({round(total_recheck * 100 / total_build_pass)})%")
      

  0%|          | 0/267239 [00:00<?, ?it/s]

Patch sets # With Failing CI jobs: 80428, Invoked Rechecks: 2307 (3)%


In [4]:
## calculate  number of them that shown gate jobs

print(total_gate_job)

690


In [8]:
import re
# count failure non voting
failure_count = 0
for change_id, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    failure = False
    
    if change_id not in pass_build_recheck_list:
        continue
    
    for ind, col in group.iterrows():
        if col['is_bot']:
            try:
                bot = re.findall(r'(failure in(?: \w+?)*s \(non\-voting\))',col['message'],re.IGNORECASE)
                for b in bot:
                    failure = True
                
            except:
                pass

    if failure:
        failure_count += 1
        
print(f"recheck failure vote with non voting: {total_recheck}, Invoked Rechecks: {failure_count} ({round(failure_count * 100 / total_recheck)})%")


  0%|          | 0/267239 [00:00<?, ?it/s]

recheck failure vote with non voting: 2307, Invoked Rechecks: 1279 (55)%


In [9]:
print(failure_count)

1279


In [12]:
for change_id, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    failure = False
    
    if change_id not in pass_build_recheck_list:
        continue
    
    
    for ind, col in group.iterrows():
        if col['is_bot']:
            try:
                bot = re.findall(r'(failure in(?: \w+?)*s \(non\-voting\))',col['message'],re.IGNORECASE)
                for b in bot:
                    failure = True
                
            except:
                pass

    if not failure:
        print(change_id)
    

  0%|          | 0/267239 [00:00<?, ?it/s]

('I00d6f13dd991074e4f45ada4d7cf4ccc0edbc7e1', 12)
('I00dda298f499b241a224c19657ebbb8a9015eb4d', 2)
('I014cf0f9c6c19e900d3c210a7f52b4e941bc46e7', 13)
('I015d823f9574002bf5ad118668d50e09ce0ba465', 4)
('I01635a4f492adc0f7d14747c1037a05fa7f22fd3', 4)
('I019e88fabd1d386c0d6395a7b1969315873485fd', 15)
('I01e883803d00bf30e63b2e84526b7aa625b9c74f', 1)
('I01f20f275bbd5451ace5c1e6f41ab38d488dae4e', 1)
('I020b78cc3ce035ff3466774d5d08198d89271117', 7)
('I02f78ee44492ea36a2ebb66211fa488034e74b5e', 1)
('I02f78ee44492ea36a2ebb66211fa488034e74b5e', 3)
('I02f78ee44492ea36a2ebb66211fa488034e74b5e', 5)
('I0330c09d72c20d63d08770b52d3071512a418260', 3)
('I036a5ceabe88dcc1fd85c09472481de7d02edf5f', 2)
('I038b49c8de0af65f38b62950b27b31d95a6301a9', 7)
('I038eeaf2508926f18b6053db0082a8aa3f3e20c6', 9)
('I055f102ee8cd726ce6e25ce868396f62d991a459', 1)
('I0589c2500f0ba7f13830921f63f77e2beefaebb4', 1)
('I05ce933e5bd01bf59a52e7d49facce26efa693aa', 6)
('I05e834b4c88250e86746ce5fe8f85b0ea9d4d011', 1)
('I061aa6f8cffb1a

KeyboardInterrupt: 

In [13]:
for change_id, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    if change_id == ('I0a250cc8a75e622f07ba9931fae0503e1421bc70', 1):
        
        for ind, col in group.iterrows():
            print(col['message'])

  0%|          | 0/267239 [00:00<?, ?it/s]

Uploaded patch set 1.
Patch Set 1: Verified+1

Build succeeded (check pipeline).

- noop noop : SUCCESS in 0s
- tox-docs http://logs.openstack.org/70/609370/1/check/tox-docs/6c23e8c/html/ : SUCCESS in 2m 30s
Patch Set 1: Code-Review+2
Patch Set 1: Workflow+1
Patch Set 1:

recheck
Patch Set 1: Verified+2

Build succeeded (gate pipeline).

- noop noop : SUCCESS in 0s
Change has been successfully merged by Zuul
Patch Set 1: -Verified

Starting gate jobs.
